In [1]:
import pandas as pd
import time
import math

#Decimal value used in similarity
decimal = 2

#movie attribute similarity threshold
simalarity_threshold = 0.85

#param for movie atributes similarity weight
genre_num = 0.75
actor_num = 0.15
director_num = 0.05
country_num = 0.05

#User attirbute sinilarity threshold
user_threshold = 0.5

#weight for for combining user and item predicted value 
user_fraction = 0.3

#file name for filled matrix 
filename = 'pivot_filled.csv'

In [2]:
def getsimilarity_fromdf(id_1,id_2,a):
    if id_1  == id_2:
        return 1.0
    
    else:
        b = a.loc[(a['From'] == id_1) & (a['To'] == id_2)]

        if b.empty:
            #print('DataFrame is empty!')
            b = a.loc[(a['From'] == id_2) & (a['To'] == id_1)]
            if b.empty:
                return 0.0
            else:
                b = b.reset_index()
                return b.at[0,'sim']

        else: 
            b = b.reset_index()
            return b.at[0,'sim']
        
        
def get_pred_rating(userid,movieid,userrated):
    
    #filter the only movie related to movieid
    temp_df = new_df2.loc[(new_df2['From'] == movieid) | (new_df2['To'] == movieid)]
    cond = temp_df.To == movieid
    temp_df.loc[cond, ['From', 'To']] = temp_df.loc[cond, ['To', 'From']].values

    #filter only user rated movie
    check_df = temp_df[temp_df['To'].isin(userrated)]
    check_df = check_df.loc[(check_df['total_sim'] > simalarity_threshold)].sort_values(by=['total_sim'], ascending=False)
    check_df = check_df.head(10)

    pred_rating_upper = 0
    pred_rating_lower = 0

    for index, row in check_df.iterrows():
        pred_rating_upper = pred_rating_upper + row['total_sim'] * get_userrating(userid,row['To'])
        pred_rating_lower = pred_rating_lower + row['total_sim']
    
    if pred_rating_lower <= 0:
        return float('nan')
    else:
        final_pred = pred_rating_upper/ pred_rating_lower
        return round(final_pred,decimal)
    
def get_usermovielist(userid):
    b = trainfile.loc[trainfile['user_id'] == userid]
    temp = b['movie_id'].tolist()    
    return temp

    
def get_userrating(userid,movieid):
    a = trainfile.loc[(trainfile['movie_id'] == movieid) & (trainfile['user_id'] == userid), 'rating']
    if a.empty:
        return 0
    else :
        return(a.values[0])

    
def get_user_pred_rating(userid,movieid,filtered_user_sim):

    user_pred_upper = 0
    user_pred_lower = 0

    for index, row in filtered_user_sim.iterrows():
        other_user_rating = get_userrating(row['Person2'],movieid)
        if other_user_rating > 0:
            user_pred_upper = user_pred_upper + other_user_rating * row['similarity']
            user_pred_lower = user_pred_lower + row['similarity']

    if user_pred_lower <= 0:
        return float('nan')
    else:
        final_user_rated = user_pred_upper/ user_pred_lower
        return round(final_user_rated,2)
        
def get_filtered_user_sim(userid):
    filtered_user_sim = user_sim.loc[(user_sim['Person1'] == userid) | (user_sim['Person2'] == userid)]
    filtered_user_sim = filtered_user_sim.loc[(filtered_user_sim['similarity'] >= user_threshold)]
    cond2 = filtered_user_sim.Person2 == userid
    filtered_user_sim.loc[cond2, ['Person1', 'Person2']] = filtered_user_sim.loc[cond2, ['Person2', 'Person1']].values
    filtered_user_sim = filtered_user_sim.drop_duplicates(subset=None, keep='first')
    
    return filtered_user_sim
    
        
def get_final_pred_rating(userid,movieid,userrated,filtered_user_sim):
    
    user_pred_rating = get_user_pred_rating(userid,movieid,filtered_user_sim)
    movie_pred_rating = get_pred_rating(userid,movieid,userrated)
    
    if math.isnan(user_pred_rating) & math.isnan(movie_pred_rating):
        return float('nan')
    elif math.isnan(user_pred_rating):
        return movie_pred_rating
    elif math.isnan(movie_pred_rating):
        return user_pred_rating
    else:
        return (user_pred_rating * user_fraction) +  (movie_pred_rating * (1-user_fraction))
    


In [3]:
allgenre  = pd.read_csv('processed-data/similarity-calculated/allgenre2.csv')
allactor = pd.read_csv('processed-data/similarity-calculated/allactor2.csv')
alldirector = pd.read_csv('processed-data/similarity-calculated/alldirector2.csv')
allcountry = pd.read_csv('processed-data/similarity-calculated/allcountry2.csv')
user_sim = pd.read_csv('processed-data/similarity-calculated/user_sim.csv')
trainfile = pd.read_csv('processed-data/train_file.csv')

In [5]:
new_df = allgenre.merge(allactor,  on=['From','To'])
new_df = new_df.merge(alldirector,  on=['From','To'])
new_df = new_df.merge(allcountry,  on=['From','To'])
new_df

,From,To,genre_sim,actor_sim,director_sim,country_sim
0,1,2,0.142857,0.0,0.0,0.5
1,3,1,0.200000,0.0,0.0,1.0
2,3,2,0.000000,0.0,0.0,0.5
3,3,4,0.333333,0.0,0.0,1.0
4,3,5,0.000000,0.0,0.0,1.0
...,...,...,...,...,...,...
1369967,1681,1,0.166667,0.0,0.0,1.0
1369968,1681,2,0.000000,0.0,0.0,0.5
1369969,1681,1682,0.000000,0.0,0.0,0.0
1369970,1682,1,0.000000,0.0,0.0,0.0


In [6]:
new_df2 = new_df

In [7]:
new_df2['total_sim'] = new_df.apply(lambda row: 
                                    (genre_num*row.genre_sim) + (actor_num*row.actor_sim) + (director_num*row.director_sim) + (country_num*row.country_sim) , axis=1)

In [8]:
new_df2

,From,To,genre_sim,actor_sim,director_sim,country_sim,total_sim
0,1,2,0.142857,0.0,0.0,0.5,0.132143
1,3,1,0.200000,0.0,0.0,1.0,0.200000
2,3,2,0.000000,0.0,0.0,0.5,0.025000
3,3,4,0.333333,0.0,0.0,1.0,0.300000
4,3,5,0.000000,0.0,0.0,1.0,0.050000
...,...,...,...,...,...,...,...
1369967,1681,1,0.166667,0.0,0.0,1.0,0.175000
1369968,1681,2,0.000000,0.0,0.0,0.5,0.025000
1369969,1681,1682,0.000000,0.0,0.0,0.0,0.000000
1369970,1682,1,0.000000,0.0,0.0,0.0,0.000000


In [9]:
trainfile.sort_values(by=['movie_id','user_id'])
trainfile.drop_duplicates(subset=None, keep='first', inplace=True)

In [10]:
pivottable = trainfile.pivot(index = 'movie_id', columns ='user_id', values = 'rating')
pivottable2 = pivottable
pivottable

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
movie_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,4.0,4.0,NaN,NaN,NaN,4.0,...,2.0,NaN,4.0,NaN,4.0,NaN,NaN,5.0,NaN,NaN
2,3.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,4.0,...,5.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
5,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1673,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1674,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1676,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
movielist = list(pivottable.index) 
userlist = list(pivottable.columns)

In [12]:
#MULTIPROCESSSING -- ONLY WORK IN LINUX
import multiprocessing
from multiprocessing import Process, Manager
import numpy as np

print("Number of cpu : ", multiprocessing.cpu_count())


start_time = time.time()

narray=[]
narray = np.array(userlist)

n_thread = 8

split_array = np.array_split(narray, n_thread)

def fill_matrix(mod_userlist,new_df2,ns): 
    pivottable = ns.df
    for u in mod_userlist:
        userrated = get_usermovielist(u)
    
        for m in movielist:
            if(math.isnan(pivottable.at[m,u])):
                pivottable.at[m,u] = get_final_pred_rating(u,m,userrated,new_df2)

        print(u)
        print("--- %s seconds ---" % (time.time() - start_time))
    ns.df = pivottable
            
        
rdf = []
pivot_df =[]

for i in range(0,n_thread):
    rdf.append(new_df2.loc[(new_df2['total_sim'] > simalarity_threshold)].sort_values(by=['total_sim'], ascending=False).copy())
    pivot_df.append(pivottable[split_array[i].tolist()].copy())
#split for different process
        
if __name__ == "__main__": 
    
    
    mgr = Manager()
    
    nss = []
    procs = []
    

    for i in range(0,n_thread):

        ns = mgr.Namespace()
        ns.df = pivot_df[i]#handle golbal varieble
        nss.append(ns)
        
        proc = Process(target=fill_matrix, args=(split_array[i].tolist(),rdf[i],nss[i])) 
        procs.append(proc)
        proc.start()
        
    for proc in procs:
        proc.join()
        
    #merge back pivottable
    pivottable = pd.merge(nss[0].df, nss[1].df, right_index=True, left_index=True)
    for i in range(2,n_thread):
        pivottable = pd.merge(pivottable, nss[i].df, right_index=True, left_index=True)
    
  
    # ALl threads completely executed 
    print("Done!") 
    
    print("--- %s seconds ---" % (time.time() - start_time))

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


237
--- 244.12481498718262 seconds ---
473
--- 245.41907358169556 seconds ---
709
--- 251.13480973243713 seconds ---
1
--- 320.12241077423096 seconds ---
474
--- 457.24809861183167 seconds ---
238
--- 482.1878561973572 seconds ---
710
--- 483.57606291770935 seconds ---
2
--- 633.8092653751373 seconds ---
475
--- 698.6492350101471 seconds ---
711
--- 705.0066056251526 seconds ---
239
--- 714.2123463153839 seconds ---
712
--- 947.4242162704468 seconds ---
476
--- 951.8876678943634 seconds ---
240
--- 970.9897499084473 seconds ---
3
--- 1113.742203950882 seconds ---
713
--- 1186.2836754322052 seconds ---
477
--- 1190.7461280822754 seconds ---
241
--- 1207.5525286197662 seconds ---
714
--- 1396.4979453086853 seconds ---
242
--- 1564.9237580299377 seconds ---
715
--- 1597.6620607376099 seconds ---
478
--- 1646.9149327278137 seconds ---
716
--- 1799.2589299678802 seconds ---
4
--- 1852.3516087532043 seconds ---
479
--- 1858.0043802261353 seconds ---
243
--- 1938.3815290927887 seconds ---
717

537
--- 17329.049936294556 seconds ---
293
--- 17383.139815807343 seconds ---
778
--- 17520.62688755989 seconds ---
538
--- 17565.70361852646 seconds ---
294
--- 17616.032033920288 seconds ---
779
--- 17759.390734910965 seconds ---
539
--- 17800.959138154984 seconds ---
26
--- 17904.75781607628 seconds ---
295
--- 17985.30037546158 seconds ---
780
--- 17993.20253276825 seconds ---
540
--- 18033.30811882019 seconds ---
296
--- 18213.769327878952 seconds ---
781
--- 18232.350652694702 seconds ---
541
--- 18264.43252325058 seconds ---
297
--- 18440.081478357315 seconds ---
782
--- 18451.033415317535 seconds ---
542
--- 18492.988619327545 seconds ---
298
--- 18672.118292331696 seconds ---
783
--- 18691.24614548683 seconds ---
543
--- 18721.190130233765 seconds ---
27
--- 18806.131808757782 seconds ---
299
--- 18884.305669784546 seconds ---
784
--- 18924.21205806732 seconds ---
544
--- 18953.561496973038 seconds ---
300
--- 19115.787532567978 seconds ---
785
--- 19156.21216082573 seconds --

353
--- 34474.2724878788 seconds ---
602
--- 34478.792906045914 seconds ---
845
--- 34652.641202926636 seconds ---
354
--- 34689.50714921951 seconds ---
603
--- 34698.625940561295 seconds ---
846
--- 34839.789563179016 seconds ---
47
--- 34900.82870340347 seconds ---
355
--- 34912.42907881737 seconds ---
604
--- 34920.954209566116 seconds ---
847
--- 35051.540759563446 seconds ---
356
--- 35132.434561252594 seconds ---
605
--- 35134.762233018875 seconds ---
848
--- 35261.51042795181 seconds ---
606
--- 35333.72773647308 seconds ---
357
--- 35349.27385902405 seconds ---
849
--- 35481.71162366867 seconds ---
607
--- 35552.91002511978 seconds ---
358
--- 35625.56523895264 seconds ---
850
--- 35697.27736663818 seconds ---
608
--- 35774.66574382782 seconds ---
359
--- 35844.584810733795 seconds ---
48
--- 35891.44910597801 seconds ---
851
--- 35899.54148316383 seconds ---
609
--- 36018.45809721947 seconds ---
360
--- 36107.578707933426 seconds ---
852
--- 36112.395956754684 seconds ---
610


668
--- 48768.19523692131 seconds ---
912
--- 48960.310518980026 seconds ---
669
--- 49007.096799612045 seconds ---
70
--- 49008.34008979797 seconds ---
412
--- 49123.93007683754 seconds ---
913
--- 49166.92958045006 seconds ---
670
--- 49223.38040447235 seconds ---
413
--- 49373.92044091225 seconds ---
914
--- 49390.91973614693 seconds ---
671
--- 49440.70881319046 seconds ---
414
--- 49597.9305369854 seconds ---
915
--- 49612.039479494095 seconds ---
672
--- 49663.066366910934 seconds ---
71
--- 49692.52954077721 seconds ---
916
--- 49809.60086894035 seconds ---
415
--- 49880.13010764122 seconds ---
673
--- 49886.0307366848 seconds ---
917
--- 50030.399075746536 seconds ---
416
--- 50058.895802497864 seconds ---
72
--- 50108.77937293053 seconds ---
674
--- 50137.56544685364 seconds ---
918
--- 50244.43276596069 seconds ---
417
--- 50285.12159180641 seconds ---
675
--- 50356.05011534691 seconds ---
919
--- 50441.54094362259 seconds ---
676
--- 50565.19441509247 seconds ---
418
--- 506

152
--- 64692.053010463715 seconds ---
153
--- 64822.780353069305 seconds ---
154
--- 64951.566895246506 seconds ---
155
--- 65019.62060689926 seconds ---
156
--- 65104.24037361145 seconds ---
157
--- 65200.89723825455 seconds ---
158
--- 65271.25709891319 seconds ---
159
--- 65441.69295120239 seconds ---
160
--- 65545.1129848957 seconds ---
161
--- 65606.55496478081 seconds ---
162
--- 65665.61374902725 seconds ---
163
--- 65736.79963731766 seconds ---
164
--- 65803.65645623207 seconds ---
165
--- 65883.41084051132 seconds ---
166
--- 65955.57855772972 seconds ---
167
--- 66018.92289733887 seconds ---
168
--- 66082.64250802994 seconds ---
169
--- 66165.52232956886 seconds ---
170
--- 66268.65043258667 seconds ---
171
--- 66363.6377389431 seconds ---
172
--- 66426.93965101242 seconds ---
173
--- 66516.6129732132 seconds ---
174
--- 66573.58050990105 seconds ---
175
--- 66632.91713809967 seconds ---
176
--- 66691.38981866837 seconds ---
177
--- 66749.84298324585 seconds ---
178
--- 6684

In [13]:
pivottable

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
movie_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,2.5,3.75,3.88,4.00,4.0,4.0,4.00,3.98,4.0,...,2.0,4.0,4.00,3.0,4.0,3.75,4.2,5.0,3.67,3.25
2,3.0,NaN,3.33,3.00,3.00,4.0,NaN,2.67,2.75,1.0,...,4.0,NaN,3.00,3.0,NaN,NaN,3.5,3.0,3.00,5.00
3,4.0,NaN,2.50,3.45,4.00,NaN,3.0,2.00,1.60,2.0,...,3.0,NaN,2.00,4.0,NaN,NaN,1.0,4.0,NaN,4.00
4,3.0,2.0,3.33,3.42,3.33,NaN,5.0,3.20,3.70,4.0,...,5.0,NaN,3.33,4.5,3.0,3.40,2.0,NaN,NaN,NaN
5,3.0,2.0,3.00,3.00,3.33,4.5,3.5,2.83,3.00,NaN,...,3.5,NaN,4.00,1.0,NaN,NaN,3.0,NaN,4.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1673,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1674,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1676,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
pivottable.to_csv(filename)

##  SVD Process

By surprise library

In [10]:
pivottable.index.name = 'userId'
pivottable = pivottable.rename_axis('movieId', axis=1)
pivottable = pivottable.rename(columns={'movie_id': 'userId'})
pivottable

movieId,userId,1,2,3,4,5,6,7,8,9,...,934,935,936,937,938,939,940,941,942,943
userId,,,,,,,,,,,,,,,,,,,,,
0,1,5.0,NaN,NaN,NaN,4.0,4.0,NaN,NaN,NaN,...,2.0,NaN,4.0,NaN,4.0,NaN,NaN,5.0,NaN,NaN
1,2,3.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
2,3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,3.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
4,5,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1601,1673,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1602,1674,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1603,1676,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
u = pivottable.melt(id_vars=['userId'])
u2 = u[u['value'].notna()]
u2.to_csv(r'processed-data/temp.csv', header=False, sep=';', index=False)

In [25]:
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import PredefinedKFold
from surprise.model_selection import cross_validate
from surprise import SVDpp
from surprise import NMF
from surprise import SVD

import numpy as np
import pandas as pd

reader = Reader(name=None, line_format=u'user item rating', sep=';', rating_scale=(1, 5), skip_lines=0)

folds_files = [('processed-data/temp.csv','processed-data/test_file.csv')]
data = Dataset.load_from_folds(folds_files, reader=reader)
pkf = PredefinedKFold()

algo = SVDpp() #or "SVD"

for trainset, testset in pkf.split(data):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)
    accuracy.mae(predictions, verbose=True)

RMSE: 0.9292
MAE:  0.7316


In [ ]:
testtable = trainfile.pivot(index = 'movie_id', columns ='user_id', values = 'rating')
pivotedit = pivottable.drop(columns='userId')

print("Original rating count:       ",testtable.count().sum())
print("Filled-matrix rating count:  ",pivotedit.count().sum())
print("Total filled rating count:   ",pivotedit.count().sum() - testtable.count().sum())
print()
print("Original matrix sparsity:    ",testtable.isnull().sum().sum()/(testtable.shape[0]*testtable.shape[1]))
print("Filled matrix sparsity:      ",pivotedit.isnull().sum().sum()/(pivotedit.shape[0]*pivotedit.shape[1]))